# Programming Assignment: 
## Готовим LDA по рецептам

In [1]:
import numpy
numpy.__version__

'1.11.1'

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули *json* и *gensim*. Первый входит в дистрибутив Anaconda, второй можно поставить командой 

*pip install gensim*

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.

### Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [2]:
import json

In [3]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [4]:
print recipes[0]

{u'cuisine': u'greek', u'id': 10259, u'ingredients': [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']}


### Составление корпуса

In [5]:
from gensim import corpora, models
import numpy as np

Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов: 

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.

In [6]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  # составляем корпус документов

In [79]:
dictionary.id2token[116]

u'cooking spray'

In [7]:
print texts[0]
print corpus[0]

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


У объекта dictionary есть две полезных переменных: *dictionary.id2token* и *dictionary.token2id*; эти словари позволяют находить соответствие между ингредиентами и их индексами.

### Обучение модели
Вам может понадобиться [документация](https://radimrehurek.com/gensim/models/ldamodel.html) LDA в gensim.

__Задание 1.__ Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию. 


Затем вызовите метод модели *show_topics*, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода *show_topics* указать параметр *formatted=True*, то топы ингредиентов будет удобно выводить на печать, если *formatted=False*, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе __не нужно__ учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. __Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.__

In [10]:
np.random.seed(76543)
# здесь код для построения модели:
from gensim.models.ldamodel import LdaModel
%time lda = LdaModel(corpus, num_topics=40, passes=5)

CPU times: user 2min 52s, sys: 1.08 s, total: 2min 53s
Wall time: 3min 33s


In [19]:
topics = lda.show_topics(num_topics=40, num_words=10, formatted=False)

In [34]:
topics_tokens = [(topic[0], [(dictionary.id2token[int(token[0])], token[1]) for token in topic[1]]) for topic in topics]

In [35]:
topics_tokens

[(0,
  [(u'cooking spray', 0.08893168359335854),
   (u'salt', 0.083023313122701475),
   (u'garlic cloves', 0.079694482383533466),
   (u'olive oil', 0.068056697732332513),
   (u'chopped onion', 0.065740286303090811),
   (u'crushed red pepper', 0.036603425389428061),
   (u'fat free less sodium chicken broth', 0.036151588839948165),
   (u'black pepper', 0.034352059579260526),
   (u'ground black pepper', 0.032028828039852991),
   (u'water', 0.032026346546356499)]),
 (1,
  [(u'carrots', 0.084685047417615594),
   (u'onions', 0.0594771859048897),
   (u'sauce', 0.056696097267259209),
   (u'cabbage', 0.056348277206384002),
   (u'spinach', 0.046496298177981413),
   (u'beef', 0.039387696216511205),
   (u'low sodium chicken broth', 0.033278198173197153),
   (u'water', 0.032572054032540064),
   (u'firm tofu', 0.029467834271367253),
   (u'green cabbage', 0.02771940582568299)]),
 (2,
  [(u'cracked black pepper', 0.065793037833935572),
   (u'dry red wine', 0.058809351108989154),
   (u'shortening', 0.0

In [42]:
from itertools import chain
flat_list = list(chain(*[[token[0] for token in topic[1]] for topic in topics_tokens]))

In [45]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))

In [48]:
ingredients = "salt", "sugar", "water", "mushrooms", "chicken", "eggs"
save_answers1(*[flat_list.count(i) for i in ingredients])

### Фильтрация словаря
В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [155]:
import copy
dictionary2 = copy.deepcopy(dictionary)

__Задание 2.__ У объекта dictionary2 есть переменная *dfs* — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря *filter_tokens*, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (иными словами, сумма длин всех документов коллекции) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.

In [156]:
dict_size_before = len(dictionary2)

In [157]:
stop_words = [k for k in dictionary2.dfs if dictionary2.dfs[k] > 4000]

In [158]:
dictionary2.filter_tokens(stop_words)

In [159]:
dict_size_after = len(dictionary2)

In [160]:
dict_size_before, dict_size_after, len(stop_words)

(6714, 6702, 12)

In [161]:
corpus2 = [dictionary2.doc2bow(text) for text in texts]

In [162]:
def corpus_size(corpus):
    size = 0
    for sentence in corpus:
        for word, count in sentence:
            size += count
    return size

In [163]:
corpus_size(corpus), corpus_size(corpus2)

(428275, 343686)

In [71]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [72]:
save_answers2(dict_size_before, dict_size_after, corpus_size(corpus),  corpus_size(corpus2))

### Сравнение когерентностей
__Задание 3.__ Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом *top_topics* модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [73]:
np.random.seed(76543)
%time lda2 = LdaModel(corpus2, num_topics=40, passes=5)

CPU times: user 2min 32s, sys: 1.11 s, total: 2min 33s
Wall time: 2min 35s


In [75]:
coh = lda.top_topics(corpus)
coh2 = lda2.top_topics(corpus2)

In [89]:
avg_coh = 0
cnt = 0
for topic, coherence in coh:
    cnt += 1
    avg_coh += coherence
avg_coh /= cnt

In [90]:
cnt = 0
avg_coh2 = 0
for topic, coherence in coh2:
    cnt += 1
    avg_coh2 += coherence
avg_coh2 /= cnt

In [91]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

In [92]:
print(avg_coh, avg_coh2)
save_answers3(avg_coh, avg_coh2)

(-627.26151033997087, -682.55267319317477)


Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло. 

### Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу. 

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом *get_document_topics* второй модели:

In [112]:
lda2.get_document_topics(corpus2[0])

[(4, 0.12812499999999988),
 (15, 0.32363797161919383),
 (22, 0.27663627763376558),
 (37, 0.15910075074703992)]

Также выведите содержимое переменной *.alpha* второй модели:

In [113]:
lda2.alpha

array([ 0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,
        0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,
        0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,
        0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,
        0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025,  0.025])

У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

__Задание 4.__ Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр __alpha=1__, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, __превосходящих 0.01__, в матрицах темы-документы обеих моделей. Другими словами, запросите темы  модели для каждого документа с параметром *minimum_probability=0.01* и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [103]:
np.random.seed(76543)
%time lda3 = LdaModel(corpus2, num_topics=40, alpha=1, passes=5)

CPU times: user 6min 59s, sys: 4.13 s, total: 7min 3s
Wall time: 7min 39s


In [114]:
lda3.get_document_topics(corpus[0])

[(0, 0.040816326530612242),
 (1, 0.020408163265306121),
 (2, 0.020408163265306121),
 (3, 0.040816326530612242),
 (4, 0.020408163265306121),
 (5, 0.020408163265306121),
 (6, 0.020408163265306121),
 (7, 0.020408163265306121),
 (8, 0.020408163265306121),
 (9, 0.040816326530612242),
 (10, 0.020408163265306121),
 (11, 0.040816326530612242),
 (12, 0.020408163265306121),
 (13, 0.020408163265306121),
 (14, 0.020408163265306121),
 (15, 0.020408163265306121),
 (16, 0.020408163265306121),
 (17, 0.020408163265306121),
 (18, 0.020408163265306121),
 (19, 0.020408163265306121),
 (20, 0.020408163265306121),
 (21, 0.040816326530612242),
 (22, 0.020408163265306121),
 (23, 0.040816326530612242),
 (24, 0.020408163265306121),
 (25, 0.020408163265306121),
 (26, 0.020408163265306121),
 (27, 0.020408163265306121),
 (28, 0.040816326530612235),
 (29, 0.020408163265306121),
 (30, 0.020408163265306121),
 (31, 0.040816326530612242),
 (32, 0.020408163265306121),
 (33, 0.020408163265306121),
 (34, 0.0204081632653061

In [125]:
sz3 = 0
for text in corpus2:
    sz3 += len(lda3.get_document_topics(text, minimum_probability=0.01))

In [126]:
sz2 = 0
for text in corpus2:
    sz2 += len(lda2.get_document_topics(text, minimum_probability=0.01))

In [127]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

In [128]:
save_answers4(sz2, sz3)

Таким образом, гиперпараметр __alpha__ влияет на разреженность распределений тем в документах. Аналогично гиперпараметр __eta__ влияет на разреженность распределений слов в темах.

### LDA как способ понижения размерности
Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

__Задание 5.__ Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу $\Theta = p(t|d)$ вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

In [119]:
X = []
for text in corpus2:
    feats = [0] * 40
    for topic, value in lda2.get_document_topics(text):
        feats[topic] = value
    X.append(feats)
X = numpy.array(X)

In [129]:
y = [recipe[u'cuisine'] for recipe in recipes]

In [140]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
import numpy

In [137]:
forest = RandomForestClassifier(n_estimators=100, n_jobs=4)
score = cross_val_score(forest, X, y, cv=3)


In [141]:
accuracy = numpy.mean(score)

In [138]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

In [142]:
save_answers5(accuracy)

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.

### LDA — вероятностная модель
Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа $d$ длины $n_d$:
1. Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: $\theta_d \sim Dirichlet(\alpha)$
1. Для каждого слова $w = 1, \dots, n_d$:
    1. Сгенерировать тему из дискретного распределения $t \sim \theta_{d}$
    1. Сгенерировать слово из дискретного распределения $w \sim \phi_{t}$.
    
Подробнее об этом в [Википедии](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation).

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)

In [164]:
def generate_recipe(model, num_ingredients):
    theta = np.random.dirichlet(model.alpha)
    for i in range(num_ingredients):
        t = np.random.choice(np.arange(model.num_topics), p=theta)
        topic = model.show_topic(0, topn=model.num_terms)
        topic_distr = [x[1] for x in topic]
        terms = [x[0] for x in topic]
        w = np.random.choice(terms, p=topic_distr)
        print w

In [175]:
generate_recipe(lda2, 10)

6263
1184
269
5575
713
11
5333
269
6064
6154


### Интерпретация построенной модели
Вы можете рассмотреть топы ингредиентов каждой темы. Большиснтво тем сами по себе похожи на рецепты; в некоторых собираются продукты одного вида, например, свежие фрукты или разные виды сыра.

Попробуем эмпирически соотнести наши темы с национальными кухнями (cuisine). Построим матрицу $A$ размера темы $x$ кухни, ее элементы $a_{tc}$ — суммы $p(t|d)$ по всем документам $d$, которые отнесены к кухне $c$. Нормируем матрицу на частоты рецептов по разным кухням, чтобы избежать дисбаланса между кухнями. Следующая функция получает на вход объект модели, объект корпуса и исходные данные и возвращает нормированную матрицу $A$. Ее удобно визуализировать с помощью seaborn.

In [174]:
import pandas
import seaborn

from matplotlib import pyplot as plt
%matplotlib inline

/Users/astepanov/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/astepanov/anaconda3/envs/ipykernel_py2/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [172]:
def compute_topic_cuisine_matrix(model, corpus, recipes):
    # составляем вектор целевых признаков
    targets = list(set([recipe["cuisine"] for recipe in recipes]))
    # составляем матрицу
    tc_matrix = pandas.DataFrame(data=np.zeros((model.num_topics, len(targets))), columns=targets)
    for recipe, bow in zip(recipes, corpus):
        recipe_topic = model.get_document_topics(bow)
        for t, prob in recipe_topic:
            tc_matrix[recipe["cuisine"]][t] += prob
    # нормируем матрицу
    target_sums = pandas.DataFrame(data=np.zeros((1, len(targets))), columns=targets)
    for recipe in recipes:
        target_sums[recipe["cuisine"]] += 1
    return pandas.DataFrame(tc_matrix.values/target_sums.values, columns=tc_matrix.columns)

In [173]:
def plot_matrix(tc_matrix):
    plt.figure(figsize=(10, 10))
    seaborn.heatmap(tc_matrix, square=True)

In [ ]:
# Визуализируйте матрицу
plot_matrix(compute_topic_cuisine_matrix(lda2, corpus2, recipes))

Чем темнее квадрат в матрице, тем больше связь этой темы с данной кухней. Мы видим, что у нас есть темы, которые связаны с несколькими кухнями. Такие темы показывают набор ингредиентов, которые популярны в кухнях нескольких народов, то есть указывают на схожесть кухонь этих народов. Некоторые темы распределены по всем кухням равномерно, они показывают наборы продуктов, которые часто используются в кулинарии всех стран. 

Жаль, что в датасете нет названий рецептов, иначе темы было бы проще интерпретировать...

### Заключение
В этом задании вы построили несколько моделей LDA, посмотрели, на что влияют гиперпараметры модели и как можно использовать построенную модель. 